In [1]:
import sys
sys.path.append("../")

In [52]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [3]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [4]:
df_raw.shape

(44055, 14)

In [5]:
df_an = df_raw.copy() #df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.shape

(44055, 14)

In [7]:
df_an = RSI(df_an)

In [8]:
df_an.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
44050,2023-01-31 19:00:00+00:00,2687,1.08722,1.08741,1.08630,1.08642,1.08715,1.08734,1.08622,1.08635,1.08728,1.08748,1.08637,1.08649,54.520710
44051,2023-01-31 20:00:00+00:00,3285,1.08643,1.08732,1.08599,1.08720,1.08636,1.08724,1.08592,1.08711,1.08650,1.08739,1.08606,1.08728,57.182766
44052,2023-01-31 21:00:00+00:00,1964,1.08720,1.08726,1.08618,1.08632,1.08712,1.08719,1.08610,1.08623,1.08727,1.08732,1.08626,1.08640,53.386083
44053,2023-01-31 22:00:00+00:00,547,1.08642,1.08646,1.08609,1.08618,1.08608,1.08627,1.08589,1.08598,1.08675,1.08677,1.08626,1.08637,52.785621
44054,2023-01-31 23:00:00+00:00,1334,1.08616,1.08657,1.08604,1.08608,1.08595,1.08650,1.08587,1.08599,1.08636,1.08666,1.08613,1.08618,52.332841


In [9]:
df_an = apply_patterns(df_an)

In [10]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [11]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [12]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c', 
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [13]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [14]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,ENGULFING,direction,EMA_200,RSI_14
0,2016-01-14 05:00:00+00:00,1.08914,1.08934,1.08763,1.08787,1.08906,1.08927,1.08755,1.08779,1.08921,1.08942,1.08771,1.08795,True,-1,1.085276,57.119596
1,2016-01-14 06:00:00+00:00,1.08784,1.08805,1.08669,1.08672,1.08776,1.08797,1.08662,1.08663,1.08791,1.08813,1.08676,1.08680,False,-1,1.085293,51.174264
2,2016-01-14 07:00:00+00:00,1.08672,1.08720,1.08544,1.08598,1.08664,1.08713,1.08537,1.08591,1.08680,1.08728,1.08551,1.08606,False,-1,1.085301,47.731446
3,2016-01-14 08:00:00+00:00,1.08594,1.09183,1.08567,1.09134,1.08588,1.09176,1.08559,1.09126,1.08601,1.09191,1.08574,1.09141,True,1,1.085370,65.720674
4,2016-01-14 09:00:00+00:00,1.09136,1.09366,1.08982,1.09022,1.09129,1.09359,1.08974,1.09015,1.09143,1.09374,1.08990,1.09028,False,-1,1.085425,60.996611


In [15]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        elif row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE

In [16]:
df_slim['SIGNAL'] = df_slim.apply(apply_signal, axis=1)

In [17]:
df_slim['SIGNAL'].value_counts()

 0    39973
-1     1949
 1     1934
Name: SIGNAL, dtype: int64

In [18]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            return (row.ask_c - row.ask_o) * PROFIT_FACTOR + row.ask_c
        else:
            return (row.bid_c - row.bid_o) * PROFIT_FACTOR + row.bid_c
    else:
        return 0.0
    
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            return row.ask_o
        else:
            return row.bid_o
    else:
        return 0.0

In [19]:
df_slim['TP'] = df_slim.apply(apply_take_profit, axis=1)
df_slim['SL'] = df_slim.apply(apply_stop_loss, axis=1)

In [20]:
df_slim[df_slim.SIGNAL == BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
3,2016-01-14 08:00:00+00:00,1.08594,1.09183,1.08567,1.09134,1.08588,1.09176,1.08559,1.09126,1.08601,1.09191,1.08574,1.09141,True,1,1.085370,65.720674,1,1.099510,1.08601
5,2016-01-14 10:00:00+00:00,1.09019,1.09289,1.08996,1.09227,1.09013,1.09283,1.08987,1.09221,1.09025,1.09295,1.09004,1.09233,True,1,1.085504,65.837097,1,1.095450,1.09025
25,2016-01-15 06:00:00+00:00,1.08788,1.08898,1.08758,1.08872,1.08781,1.08890,1.08751,1.08864,1.08796,1.08905,1.08766,1.08881,True,1,1.085911,55.194357,1,1.090085,1.08796
27,2016-01-15 08:00:00+00:00,1.08685,1.09046,1.08634,1.09007,1.08678,1.09039,1.08626,1.09000,1.08692,1.09054,1.08642,1.09014,True,1,1.085968,58.204010,1,1.094970,1.08692
35,2016-01-15 16:00:00+00:00,1.09510,1.09680,1.09420,1.09635,1.09502,1.09670,1.09409,1.09628,1.09517,1.09693,1.09427,1.09642,True,1,1.086589,69.048533,1,1.098295,1.09517


In [21]:
df_plot = df_slim.iloc[140:220]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))
    
cp.show_plot(line_traces=['EMA_200'], sec_traces=['RSI_14'], height=250)

In [23]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        
        if row.SIGNAL == BUY:
            self.start_price = row.ask_c
            self.trigger_price = row.ask_c
        else:
            self.start_price = row.bid_c
            self.trigger_price = row.bid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
    
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
    
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.ask_h)
    

In [24]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))

In [25]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [26]:
df_results.result.sum()

-513.0

In [27]:
df_results.sort_values(by="start_index", inplace=True)

In [28]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [29]:
df_m5.shape

(526969, 14)

In [30]:
df_m5_slim = df_m5[['time', 'bid_h', 'bid_l', 'ask_h', 'ask_l']].copy()

In [31]:
df_m5_slim.head()

,time,bid_h,bid_l,ask_h,ask_l
0,2016-01-03 22:00:00+00:00,1.08699,1.08674,1.08796,1.08774
1,2016-01-03 22:05:00+00:00,1.08710,1.08689,1.08770,1.08728
2,2016-01-03 22:10:00+00:00,1.08696,1.08643,1.08734,1.08705
3,2016-01-03 22:15:00+00:00,1.08665,1.08644,1.08723,1.08694
4,2016-01-03 22:20:00+00:00,1.08689,1.08646,1.08745,1.08695


In [32]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [33]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [34]:
df_signals['start_index_h1'] = df_signals.index

In [35]:
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,...,ask_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL,m5_start,start_index_h1
3,2016-01-14 08:00:00+00:00,1.08594,1.09183,1.08567,1.09134,1.08588,1.09176,1.08559,1.09126,1.08601,...,1.09141,True,1,1.085370,65.720674,1,1.099510,1.08601,2016-01-14 09:00:00+00:00,3
5,2016-01-14 10:00:00+00:00,1.09019,1.09289,1.08996,1.09227,1.09013,1.09283,1.08987,1.09221,1.09025,...,1.09233,True,1,1.085504,65.837097,1,1.095450,1.09025,2016-01-14 11:00:00+00:00,5
25,2016-01-15 06:00:00+00:00,1.08788,1.08898,1.08758,1.08872,1.08781,1.08890,1.08751,1.08864,1.08796,...,1.08881,True,1,1.085911,55.194357,1,1.090085,1.08796,2016-01-15 07:00:00+00:00,25
27,2016-01-15 08:00:00+00:00,1.08685,1.09046,1.08634,1.09007,1.08678,1.09039,1.08626,1.09000,1.08692,...,1.09014,True,1,1.085968,58.204010,1,1.094970,1.08692,2016-01-15 09:00:00+00:00,27
35,2016-01-15 16:00:00+00:00,1.09510,1.09680,1.09420,1.09635,1.09502,1.09670,1.09409,1.09628,1.09517,...,1.09642,True,1,1.086589,69.048533,1,1.098295,1.09517,2016-01-15 17:00:00+00:00,35


In [36]:
df_signals.columns

Index(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h', 'bid_l',
       'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'ENGULFING', 'direction',
       'EMA_200', 'RSI_14', 'SIGNAL', 'TP', 'SL', 'm5_start',
       'start_index_h1'],
      dtype='object')

In [37]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h', 'bid_l',
        'ask_o', 'ask_h', 'ask_l', 'ENGULFING', 'direction',
       'EMA_200', 'RSI_14'], axis=1, inplace=True)

In [40]:
df_signals.head()

,start_price_BUY,start_price_SELL,SIGNAL,TP,SL,time,start_index_h1
3,1.09126,1.09141,1,1.099510,1.08601,2016-01-14 09:00:00+00:00,3
5,1.09221,1.09233,1,1.095450,1.09025,2016-01-14 11:00:00+00:00,5
25,1.08864,1.08881,1,1.090085,1.08796,2016-01-15 07:00:00+00:00,25
27,1.09000,1.09014,1,1.094970,1.08692,2016-01-15 09:00:00+00:00,27
35,1.09628,1.09642,1,1.098295,1.09517,2016-01-15 17:00:00+00:00,35


In [39]:
df_signals.rename(columns={
    'bid_c': 'start_price_BUY',
    'ask_c': 'start_price_SELL',
    'm5_start': 'time'
}, inplace=True)

In [41]:
df_signals.head(2)

,start_price_BUY,start_price_SELL,SIGNAL,TP,SL,time,start_index_h1
3,1.09126,1.09141,1,1.09951,1.08601,2016-01-14 09:00:00+00:00,3
5,1.09221,1.09233,1,1.09545,1.09025,2016-01-14 11:00:00+00:00,5


In [42]:
df_m5_slim.head(2)

,time,bid_h,bid_l,ask_h,ask_l
0,2016-01-03 22:00:00+00:00,1.08699,1.08674,1.08796,1.08774
1,2016-01-03 22:05:00+00:00,1.08710,1.08689,1.08770,1.08728


In [43]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [44]:
merged.fillna(0, inplace=True)

In [45]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [46]:
merged.head()

,time,bid_h,bid_l,ask_h,ask_l,start_price_BUY,start_price_SELL,SIGNAL,TP,SL,start_index_h1
0,2016-01-03 22:00:00+00:00,1.08699,1.08674,1.08796,1.08774,0.0,0.0,0,0.0,0.0,0
1,2016-01-03 22:05:00+00:00,1.08710,1.08689,1.08770,1.08728,0.0,0.0,0,0.0,0.0,0
2,2016-01-03 22:10:00+00:00,1.08696,1.08643,1.08734,1.08705,0.0,0.0,0,0.0,0.0,0
3,2016-01-03 22:15:00+00:00,1.08665,1.08644,1.08723,1.08694,0.0,0.0,0,0.0,0.0,0
4,2016-01-03 22:20:00+00:00,1.08689,1.08646,1.08745,1.08695,0.0,0.0,0,0.0,0.0,0


In [47]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        
        if row.SIGNAL == BUY:
            self.start_price = row.start_price_BUY
            self.trigger_price = row.start_price_BUY
        else:
            self.start_price = row.start_price_SELL
            self.trigger_price = row.start_price_SELL
            
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1
    
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
    
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.ask_h)
    

In [48]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))
        
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

In [49]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [50]:
df_res_m5.head()

,running,start_index_m5,start_index_h1,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,2459,5,1.09221,1.09013,1,1.095450,1.09025,-1.0,2016-01-14 12:30:00+00:00,2016-01-14 11:00:00+00:00,20
1,False,2435,3,1.09126,1.08558,1,1.099510,1.08601,-1.0,2016-01-14 15:40:00+00:00,2016-01-14 09:00:00+00:00,82
2,False,2699,25,1.08864,1.08796,1,1.090085,1.08796,-1.0,2016-01-15 07:15:00+00:00,2016-01-15 07:00:00+00:00,5
3,False,2723,27,1.09000,1.09560,1,1.094970,1.08692,1.5,2016-01-15 13:35:00+00:00,2016-01-15 09:00:00+00:00,57
4,False,2819,35,1.09628,1.09439,1,1.098295,1.09517,-1.0,2016-01-15 17:55:00+00:00,2016-01-15 17:00:00+00:00,13


In [51]:
df_res_m5.result.sum()

-1121.5